<a href="https://colab.research.google.com/github/umangja/Morph_Inflection_Generation/blob/master/MorphInflectionGeneration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Importing important library**


In [0]:
import pandas as pd
import matplotlib.pyplot as plt
import random
import torch 
from torch import nn
from torch.autograd import Variable
from torch.nn import functional as F
import time
import math



# Set the random seeds for reproducability.

In [0]:
SEED = 1
random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')



# Processing file and other helper functions

In [0]:

"""**Processing Files And Other Useful Function**"""

def file_preprocess(file_name):
    
    # from google.colab import files
    # uploaded = files.upload()
    
    pandas_holder = pd.read_csv(file_name, sep='\t', names=['word', 'inflection', 'feature'])
    
    words_list = [list(word) for word in list(pandas_holder['word'])]
    inflection_list = [list(inflection) for inflection in list(pandas_holder['inflection'])]
    feature_list = [feat.split(';') for feat in list(pandas_holder['feature'])]
    #print(words_list);

    vocab_chars_word = [char for word in words_list for char in word]
    vocab_chars_inflection = [char for word in inflection_list for char in word]
    # print(vocab_chars_word)

    char_vocab = ['<pad>', '<sos>', '<eos>'] + list(set(vocab_chars_word+vocab_chars_inflection))
    feat_vocab = ['<pad>'] + list(set([f for feat in feature_list for f in feat]))

    # print(char_vocab)
    char_to_index = dict(zip(char_vocab, list(range(len(char_vocab)))))
    index_to_char = dict(zip(list(range(len(char_vocab))), char_vocab))
    feature_to_index = dict(zip(feat_vocab, list(range(len(feat_vocab)))))

    # 2 added here b/c sos and eos appended to each word and inflection word
    word_list_length = [len(word)+2 for word in words_list]
    inflection_list_length = [len(word)+2 for word in inflection_list]
    feature_list_length = [len(feat) for feat in feature_list]

    max_word_length = max(word_list_length)
    max_inflection_length = max(inflection_list_length)
    max_feature_length = max(feature_list_length)

    return words_list, inflection_list, feature_list, char_to_index, index_to_char, feature_to_index, max_word_length, max_inflection_length, max_feature_length

def boundary_appender(sentence_list):
    sent_list = []
    for sentence in sentence_list:
        sentence.append('<eos>')
        sentence.insert(0,'<sos>')
        sent_list.append(sentence)
    return sent_list

def sentence_pad(sentence_list, length):
    sent_list = []
    for sentence in sentence_list:
        while len(sentence) != length:
            sentence.append(char_to_index['<pad>'])
        sent_list.append(sentence)
    return sent_list

def word_pad(word, length):
    while len(word) != length:
        word.append(char_to_index['<pad>'])
    return word

def source_taget_equal_length_maker(source, target):    
    source_lengths = [len(s) for s in source]
    target_lengths = [len(t) for t in target]
    max_lengths = max(max(source_lengths), max(target_lengths))

    source_sent = sentence_pad(source, max_lengths)
    target_sent = sentence_pad(target, max_lengths)
    return source_sent, target_sent, max_lengths

def sentence_encoder(sentence_list, dictionary):
    sent_list = [[dictionary[word] for word in sentence]for sentence in sentence_list]
    return sent_list

def mini_batch_creator(sentence_list, batch_size):
    final = [sentence_list[i * batch_size:(i + 1) * batch_size] for i in range((len(sentence_list) + batch_size - 1) // batch_size)]
    return final
  
def ploatter(loss, acc, val_loss, val_acc):
    fig = plt.figure()

    # x axis values
    x = list(range(1,len(loss)+1))
    fig1=plt.figure(figsize=(18, 12), dpi= 80, facecolor='w', edgecolor='k')
    fig1=plt.subplot(1,2,1)

    lines_1 = plt.plot(x, loss, x, val_loss)
    l1, l2 = lines_1
    plt.setp(lines_1, linestyle='--')
    plt.setp(l1, linewidth=2, color='b', label='Train Loss')
    plt.setp(l2, linewidth=2, color='g', label='Validation Loss')

    plt.legend(loc='upper right')
    fig1.set_title('Loss/Epoch')
    fig1.set_xlabel('Epoch')
    fig1.set_ylabel('loss')

    fig2=plt.subplot(1, 2, 2)

    lines_2 = plt.plot(x, acc, x, val_acc)
    l1, l2 = lines_2
    plt.setp(lines_2, linestyle='--')
    plt.setp(l1, linewidth=2, color='b', label='Train Accuracy')
    plt.setp(l2, linewidth=2, color='g', label='Validation Accuracy')

    plt.legend(loc='upper right')
    fig2.set_title('Accuracy/Epoch')
    fig2.set_xlabel('Epoch')
    fig2.set_ylabel('Accuracy')
    
    
    fig.savefig(file_name+'.png')
    plt.show()
    

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs



# Encoder Class

In [0]:
class Encoder(nn.Module):
    def __init__(self,vocab_size,embedding_size, encoder_hid_dem,decoder_hid_dem,bidirectional,dropout):
        super().__init__()
        
        self.encoder_hid_dem  = encoder_hid_dem
        self.encoder_n_direction=1;
        self.bias = False
        self.dropout=dropout 
        if(bidirectional==True):
            self.encoder_n_direction=2;

        self.embedding_layer  = nn.Embedding(num_embeddings=vocab_size, embedding_dim=embedding_size, padding_idx=0)
        self.GRU_layer        = nn.GRU(input_size=embedding_size, hidden_size=encoder_hid_dem, batch_first=True, bidirectional=bidirectional)
        self.fc               = nn.Linear(encoder_hid_dem*self.encoder_n_direction,decoder_hid_dem)
        self.dropout          = nn.Dropout(dropout)


    def forward(self, input_word):
        # print(input_word.size())
        #[batch_size    src_sent_lent]

        embed_out = self.embedding_layer(input_word)
        #[BATCH_SIZE    src_sent_lent   embedding_dim]
        
        embed_out = F.relu(embed_out)
        embed_out = self.dropout(embed_out)

        self.batch = embed_out.size()[0]

        # hidden =  self.init_hidden()
        GRU_out,hidden = self.GRU_layer(embed_out)

        
        # print(GRU_out.size())
        # print(hidd.size())

        #[BATCH_SIZE    sec_sent_len    n_direction*hid_dem]
        #[n_layer*n_direction   batch_size    hid_dem]

        #where the first hid_dim elements in the third axis are the hidden states from the top layer forward RNN, and the last hid_dim elements are hidden states from the top layer backward RNN

        #hidden is stacked [forward_1, backward_1, forward_2, backward_2, ...]
        #hidden [-2, :, : ] is the last of the forwards RNN 
        #hidden [-1, :, : ] is the last of the backwards RNN

        GRU_out = F.relu(GRU_out)
        hidden = torch.tanh(self.fc(torch.cat((hidden[-2,:,:],hidden[-1,:,:]),dim=1)))
        
        # print(GRU_out.size())
        # print(hidden.size())

        #outputs = [batch_size    src sent len, encoder_hid_dim * n_direction]
        #hidden = [batch size, dec hid dim]
        return GRU_out,hidden

    def init_hidden(self):
        return (Variable(torch.eye(1, self.encoder_hid_dem)).unsqueeze(1).repeat(2, self.batch, 1).to(self.device))


# Attention Class

In [0]:
class Attention(nn.Module):
    def __init__(self,encoder_hid_dem,decoder_hid_dem,bidirectional):
        super().__init__()
        self.enc_hid_dim = encoder_hid_dem
        self.dec_hid_dim = decoder_hid_dem
        self.encoder_n_direction=1;
        if(bidirectional==True):
            self.encoder_n_direction=2;
            
        self.attn = nn.Linear((encoder_hid_dem * self.encoder_n_direction) + decoder_hid_dem, decoder_hid_dem)
        self.v = nn.Parameter(torch.rand(decoder_hid_dem))
        
    def forward(self, hidden, encoder_outputs):
        
        #hidden = [batch size, dec hid dim]
        #encoder_outputs = [batch_size    ,src sent len, enc hid dim * encoder_n_direction]
        
        batch_size = encoder_outputs.shape[0]
        src_len    = encoder_outputs.shape[1]
        
        hidden = hidden.unsqueeze(1).repeat(1, src_len, 1)
        
        #hidden          = [batch size, src sent len, dec hid dim]
        #encoder_outputs = [batch size, src sent len, enc hid dim * encoder_n_direction]
        
        energy = torch.tanh(self.attn(torch.cat((hidden, encoder_outputs), dim=2))) 
        #energy = [batch size, src sent len, dec hid dim]
        
        energy = energy.permute(0, 2, 1)
        #energy = [batch size, dec hid dim, src sent len]
        
        #v = [dec hid dim]
        v = self.v.repeat(batch_size, 1).unsqueeze(1)
        #v = [batch size, 1, dec hid dim]
                
        attention = torch.bmm(v, energy).squeeze(1)
        #attention= [batch size, src len]
            
        return F.softmax(attention, dim=1)



# Decoder Class

In [0]:
class Decoder(nn.Module):
    def __init__(self, decoder_hid_dem, encoder_hid_dem, vocab_size,embedding_dim,attention,decoder_input_size,linear_input_size,bidirectional,dropout):
        super().__init__()      
        self.encoder_hid_dem=encoder_hid_dem
        self.decoder_hid_dem=decoder_hid_dem
        self.attention=attention
        self.dropout = dropout
        self.output_dim = vocab_size

        self.decoder_n_direction=1;
        if(bidirectional==True):
            self.decoder_n_direction=2;
            
        self.GRU_layer_out = nn.GRU(decoder_input_size,decoder_hid_dem)
        self.out_layer = nn.Linear(in_features=linear_input_size, out_features=vocab_size)
        self.dropout = nn.Dropout(dropout)
        #self.GRU_layer_out.bias = torch.nn.Parameter(torch.zeros(decoder_input_size))

    def forward(self, feature, hidden,actual_word,encoder_outputs):

        feature = feature.unsqueeze(1)
        # print('decoder')
        # print(feature.size())
        #[batch_size    src_sent_lent=1   feat_size=6]
       
        # print(hidden.size())
        # [batch_size     dec_hid_dim]


        # print(actual_word.size())
        # [batch_size    src_sent_lent=1   embedding_dim]

        # print(encoder_outputs.size())
        # outputs = [batch_size    src sent len, encoder_hid_dim * encoder_n_directional]


        a = self.attention(hidden,encoder_outputs)
        #  print(a.size())
        # [batch_size    src_sent_len]

        a = a.unsqueeze(1)
        #a = [batch size, 1, src len] 

        weighted = torch.bmm(a,encoder_outputs)
        # print(weighted.size())
        # weighted = [batch size, 1, enc_hid_dim * encoder_n_direction]
        # if len(actual_word.size()) != 0:
        input_char = torch.cat((actual_word,feature,weighted),2) 
        # else:
        #     input_char = torch.cat((feature,weighted),2)

        input_char=input_char.permute(1,0,2)
        #  print(input_char.size())
        # [1    BATCH_SIZE      decoder_input_size]

        hidden = hidden.unsqueeze(0)
        # print(hidden.size())
        #[1 batch_size decoder_hid_dem]
       
        output, hidden = self.GRU_layer_out(input_char, hidden)

        # print(output.size())
        # [sent_len=1   batch_size  decoder_n_direction*decoder_hid_dem]
        # print(hidden.size())
        # [n_layer*n_direction    BATCH_SIZE      hid_dem]



        output = F.leaky_relu(output)
        output = self.dropout(output)

        output = torch.cat((output.squeeze(0),weighted.squeeze(1),actual_word.squeeze(1)),dim=1)
        pre_out = self.out_layer(output)
        predicted_output = F.log_softmax(pre_out, dim=1)

        # print(predicted_output.size())
        # [ batch_size vacab_size ]
        return predicted_output, hidden.squeeze(0)  

    def init_hidden(self, batch):
        return (Variable(torch.eye(1, self.decoder_hid_dem)).unsqueeze(1).repeat(1, batch, 1).to(self.device),Variable(torch.eye(1, self.decoder_hid_dem)).unsqueeze(1).repeat(1, batch, 1).to(self.device))



# Seq2Seq Class

In [0]:
class Seq2Seq(nn.Module):
    def __init__(self,encoder,decoder,device):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
    
    def forward(self,input_word,output_word,features_word,teaching_forcing_ratio,limit):
        #print(input_word)
        #print(input_word.size())
        input_word = input_word.to(self.device)
        output_word = output_word.to(self.device)
        features_word = features_word.to(self.device)
        
        batch_size= input_word.size()[0]
        if(limit==0):
            max_len   = input_word.size()[1]
        else:
            max_len   = limit
        vocabsize = self.decoder.output_dim
        
        actual_word = self.encoder.embedding_layer(torch.tensor(char_to_index['<sos>']).view(1, -1).to(self.device)).repeat(batch_size, 1, 1)
        encoder_outputs,hidden = self.encoder(input_word)
        features=features_word[:,:]
        
        predicted_word = torch.zeros(max_len,batch_size,vocabsize).to(self.device)
        
        for t in range(1,max_len):
            output,hidden=self.decoder(features, hidden,actual_word,encoder_outputs)
            #print(output.size())
            predicted_word[t] = output 
            topv, topi = output.topk(1)
            bs = topi.size()[0]
            temp2 = torch.zeros(0,1,300).to(self.device)
            for row in range(bs):
                index = topi[row][0].item()
                temp = self.encoder.embedding_layer(torch.tensor(index).view(1, -1).to(self.device))
                temp2 = torch.cat((temp2,temp))

            teacher_force = random.random() < teaching_forcing_ratio
            if teacher_force == 1:
                actual_word = self.encoder.embedding_layer(output_word[:,t]).unsqueeze(1)
            else:
                actual_word = temp2
                
        return predicted_word

# Hyperparams

In [0]:
file_name = 'hindi-train-medium'
evaluate_file = 'hindi-dev'

words_list, inflection_list, feature_list, char_to_index, index_to_char, feature_to_index, max_word_length, max_inflection_length, max_feature_length = file_preprocess(file_name)

words_list      = boundary_appender(words_list)
inflection_list = boundary_appender(inflection_list)

word_encode_list  = sentence_encoder(words_list, char_to_index)
inf_encode_list   = sentence_encoder(inflection_list, char_to_index)
feat_encode_list  = sentence_pad(sentence_encoder(feature_list, feature_to_index), max_feature_length)


test_words_list, test_inflection_list, test_feature_list, _, _, _, _, _, _ = file_preprocess(evaluate_file)
t_word_list       = boundary_appender(test_words_list)
t_inflection_list = boundary_appender(test_inflection_list)

test_word_encode_list = sentence_encoder(t_word_list, char_to_index)
test_inf_encode_list  = sentence_encoder(t_inflection_list, char_to_index)
test_feat_encode_list = sentence_pad(sentence_encoder(test_feature_list, feature_to_index), max_feature_length)


"""**Hyperparameters**"""
embedding_size   = 300
encoder_hid_dem  = 300
decoder_hid_dem  = 3*encoder_hid_dem
vocab_size       = len(char_to_index.keys())
dropout          = .75
BATCH_SIZE       = 10         
EPOCH_SIZE       = 30
learning_rate    = 0.00069
encoder_bidirectional =True
decoder_birectional   =True
teacher_forcing_ratio = .5


# embedding_size   = 300
# encoder_hid_dem  = 300
# decoder_hid_dem  = 1*encoder_hid_dem
# vocab_size       = len(char_to_index.keys())
# dropout          = .75
# BATCH_SIZE       = 10         
# EPOCH_SIZE       = 10
# learning_rate    = 0.00069
# encoder_bidirectional =True
# decoder_birectional   =True
# teacher_forcing_ratio = .5
# dropout         = 0.5


if(encoder_bidirectional):
    decoder_input_size  =len(feat_encode_list[0]) + embedding_size+(encoder_hid_dem)*2
    linear_input_size=embedding_size+decoder_hid_dem+(encoder_hid_dem)*2
else:
    decoder_input_size  =len(feat_encode_list[0]) + embedding_size+(encoder_hid_dem)*1
    linear_input_size=embedding_size+decoder_hid_dem+(encoder_hid_dem)*1
    



In [0]:
attention = Attention(encoder_hid_dem,decoder_hid_dem,encoder_bidirectional)
enc       = Encoder(vocab_size,embedding_size,encoder_hid_dem,decoder_hid_dem,encoder_bidirectional,dropout)
dec       = Decoder(decoder_hid_dem,encoder_hid_dem,vocab_size,embedding_size,attention,decoder_input_size,linear_input_size,decoder_birectional,dropout)

model     = Seq2Seq(enc,dec,device).to(device)

# Intialization of weights

In [0]:

def init_weights(m):
    for name, param in m.named_parameters():
        if 'weight' in name:
            nn.init.normal_(param.data, mean=0, std=0.01)
        else:
            nn.init.constant_(param.data, 0)
            
model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding_layer): Embedding(58, 300, padding_idx=0)
    (GRU_layer): GRU(300, 300, batch_first=True, bidirectional=True)
    (fc): Linear(in_features=600, out_features=900, bias=True)
    (dropout): Dropout(p=0.75)
  )
  (decoder): Decoder(
    (attention): Attention(
      (attn): Linear(in_features=1500, out_features=900, bias=True)
    )
    (GRU_layer_out): GRU(906, 900)
    (out_layer): Linear(in_features=1800, out_features=58, bias=True)
    (dropout): Dropout(p=0.75)
  )
)

# Counting Total Params

In [0]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

# Other Stuff

In [0]:
optimizer = torch.optim.Adam(model.parameters())
criterion = nn.NLLLoss()

# Training Loop

In [0]:
def train(model,input_word_minibatches,output_word_minibatches,feature_minibatches,optimizer,criterion,clip):
    model.train()
    epoch_loss=0
    ct=0
    for batch_idx, _ in enumerate(input_word_minibatches):
        train_word = input_word_minibatches[batch_idx]
        train_label = output_word_minibatches[batch_idx]
        train_feat = feature_minibatches[batch_idx]

        train_word, train_label, max_length = source_taget_equal_length_maker(train_word, train_label)

        train_word = torch.tensor(train_word)
        train_label = torch.tensor(train_label)
        train_feat = torch.FloatTensor(train_feat)
        
        #print(train_word.size())
        optimizer.zero_grad()

        predicted_word = model(train_word,train_label,train_feat,0.5,0)

        #trg = [trg sent len, batch size]
        #output = [trg sent len, batch size, output dim]
        train_label = train_label.permute(1,0)
        predicted_word = predicted_word[1:].view(-1,predicted_word.shape[-1])
        trg = train_label[1:].contiguous().view(-1).to(device)
        #print(trg.size())
        
        #trg = [(trg sent len - 1) * batch size]
        #output = [(trg sent len - 1) * batch size, output dim]
        
        loss = criterion(predicted_word, trg)
        
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        ct=ct+1
        epoch_loss += loss.item()
        
    return epoch_loss / ct

# Validation Loop

In [0]:
def validate(model,input_word_minibatches,output_word_minibatches,feature_minibatches,criterion):
    model.eval()
    epoch_loss=0
    with torch.no_grad():
      
          train_word = torch.tensor(input_word_minibatches)
          train_label = torch.tensor(output_word_minibatches)
          train_feat = torch.FloatTensor(feature_minibatches)

          predicted_word = model(train_word,train_label,train_feat,0,0)

          #trg = [trg sent len, batch size]
          #output = [trg sent len, batch size, output dim]
          
          train_label = output_word_minibatches.permute(1,0)
          predicted_word = predicted_word[1:].view(-1,predicted_word.shape[-1])
          trg = train_label[1:].contiguous().view(-1).to(device)

          #predicted_word = predicted_word[:,1:]
          #trg = output_word_minibatches[:,1:]

          #trg = [(trg sent len - 1) * batch size]
          #output = [(trg sent len - 1) * batch size, output dim]

          loss = criterion(predicted_word, trg)
          epoch_loss += loss.item()

    return epoch_loss

# Test Loop

In [0]:
def to_strings(predicted,target):
  topv,predicted = torch.topk(predicted,1,1)
  predicted_string = ""
  target_string    = ""
  for character in range(predicted.size()[0]):
      if index_to_char[predicted[character].item()] == '<eos>':
          break;
      predicted_string=predicted_string+index_to_char[predicted[character].item()]
      
  for character in range(target.size()[0]):
      if index_to_char[target[character].item()] == '<eos>':
          break;
      target_string=target_string+index_to_char[target[character].item()]

  return target_string,predicted_string
    

In [0]:
def test(model,test_words,test_inflection,test_feat,criterion):
    model.eval()
    epoch_loss=0
    with torch.no_grad():
      
          test_words      = torch.tensor(test_words)
          test_label      = torch.tensor(test_inflection)
          test_feat       = torch.FloatTensor(test_feat)

          predicted_word = model(test_words,test_label,test_feat,0,50)
          predicted_word = predicted_word[1:].view(-1,predicted_word.shape[-1])
 
          test_label = test_label.permute(1,0)
          test_label = torch.cat([test_label,torch.zeros([max(0,50-test_label.size()[0]),1],dtype=torch.long)],0)

          for index in range(test_label.size()[0],50):
              test_label[index][0]=char_to_index['pad']
          
          #print(test_label.size())
          trg = test_label[1:].contiguous().view(-1).to(device)
          
          #trg = [(trg sent len - 1) * batch size]
          #output = [(trg sent len - 1) * batch size, output dim]
          
          loss = criterion(predicted_word, trg)
          epoch_loss += loss.item()
          
          actual_string,predicted_string = to_strings(predicted_word,trg)
          print(actual_string)
          print(predicted_string)
          print(epoch_loss)
          acc=0
          index=0
          for charac in predicted_string:
              if index>=len(actual_string):
                break
              if(charac==actual_string[index]):
                acc=acc+1
              index=index+1;
              
          acc=acc/len(actual_string)
          print(acc)
          
    return acc
    

# Actual Training 

In [0]:

input_word_minibatches  = mini_batch_creator(word_encode_list, batch_size=BATCH_SIZE)
output_word_minibatches = mini_batch_creator(inf_encode_list, batch_size=BATCH_SIZE)
feature_minibatches     = mini_batch_creator(feat_encode_list, batch_size=BATCH_SIZE)

# valid_word_minibatches  = mini_batch_creator(valid_word,batch_size=10)
# valid_label_minibatches = mini_batch_creator(valid_label,batch_size=10)
# valid_feat_minibatches  = mini_batch_creator(valid_feat,batch_size=10)

valid_word, valid_label, max_length_1 = source_taget_equal_length_maker(test_word_encode_list, test_inf_encode_list)
valid_word = torch.tensor(valid_word)
valid_label = torch.tensor(valid_label)
valid_feat = torch.FloatTensor(test_feat_encode_list)


In [0]:
def Training():
  CLIP = 1
  best_valid_loss = float('inf')
  for epoch in range(EPOCH_SIZE):
      start_time = time.time()

      train_loss = train(model, input_word_minibatches,output_word_minibatches,feature_minibatches, optimizer, criterion, CLIP)
      valid_loss = validate(model, valid_word,valid_label,valid_feat, criterion)

      end_time = time.time()

      epoch_mins, epoch_secs = epoch_time(start_time, end_time)

      if valid_loss < best_valid_loss:
          best_valid_loss = valid_loss
          torch.save(model.state_dict(), 'model.pt')

      print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
      print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
      print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

# Printing Test

In [0]:
test_file = 'hindi-test'
test_words_list, test_inflection_lists, test_feature_list, _, _, _, _, _, _ = file_preprocess(test_file)

test_words = boundary_appender(test_words_list)
test_words = sentence_encoder(test_words, char_to_index)

test_inflection = boundary_appender(test_inflection_lists)
test_inflection = sentence_encoder(test_inflection, char_to_index)

test_feat  = sentence_pad(sentence_encoder(test_feature_list, feature_to_index), max_feature_length)


In [0]:
def Testing():
  count=0
  for index in range(len(test_words)):
      print(index)
      test_word1 = [test_words[index]]
      test_feat1 = [test_feat[index]]
      test_label1 = [test_inflection[index]]

      test_word1, test_label1, max_len = source_taget_equal_length_maker(test_word1,test_label1)
      acc = test(model,test_word1,test_label1,test_feat1,criterion)
      if acc==1:
          count=count+1
      print()
  print('exactly correct no of examples are ',count)

In [0]:
def Testing1():
  count=0
  model.load_state_dict(torch.load('model.pt'))
  for index in range(len(test_words)):
      print(index)
      test_word1 = [test_words[index]]
      test_feat1 = [test_feat[index]]
      test_label1 = [test_inflection[index]]

      test_word1, test_label1, max_len = source_taget_equal_length_maker(test_word1,test_label1)
      acc = test(model,test_word1,test_label1,test_feat1,criterion)
      if acc==1:
          count=count+1
      print()
  print('exactly correct no of examples are ',count)

#Load Model

In [0]:
# Training()


In [0]:
Testing()

In [0]:
Testing1()